In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from optimus import Optimus

# Create optimus
op = Optimus()

df = op.read.csv("data/random.csv",header=True, sep=";")

C:\Users\argenisleon\Anaconda3\lib\site-packages\socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable

    You are using PySparkling of version 2.4.10, but your PySpark is of
    version 2.3.1. Please make sure Spark and PySparkling versions are compatible. 


In [2]:
df.table()

## Key Collision

In [3]:
from optimus.ml import keycollision as keyCol

# op.spark.conf.set("spark.sql.shuffle.partitions", "4")
# op.spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 1 * 1024 * 1024 * 1024)

keyCol.fingerprint(df, 'STATE').table(columns=["STATE","STATE_FINGERPRINT"])

STATE 1 (string) nullable,STATE_FINGERPRINT 2 (string) nullable
Distrito⋅Federal,distritofederal
Distrito⋅Federal,distritofederal
Distrito⋅Federal,distritofederal
Distrito⋅Federal,distritofederal
Distrito⋅Federal,distritofederal
Distrito⋅Federal,distritofederal
Distrito⋅Federal,distritofederal
Distrito⋅Federal,distritofederal
Distrito⋅Federal,distritofederal
Distrito⋅Federal,distritofederal


In [4]:
keyCol.fingerprint_cluster(df, 'STATE').table()

cluster_size 1 (int) not nullable,cluster 2 (array<string>) nullable,count 3 (bigint) nullable,recommended 4 (string) nullable
1,['Estado⋅de⋅México'],810,Estado⋅de⋅México
2,"['México⋅D.F.',⋅'Mexico⋅D.F.']",2495,Mexico⋅D.F.
1,['D.F.'],66,D.F.
1,['Distriro⋅Federal'],259,Distriro⋅Federal
3,"['Distrito⋅Federal',⋅'DISTRITO⋅FEDERAL',⋅'distrito⋅federal']",11930,Distrito⋅Federal


In [5]:
keyCol.n_gram_fingerprint(df, 'STATE', 2).table(columns=["STATE","STATE_NGRAM", "STATE_NGRAM_FINGERPRINT"])

STATE 1 (string) nullable,STATE_NGRAM 2 (array<string>) not nullable,STATE_NGRAM_FINGERPRINT 3 (string) nullable
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],


In [27]:
keyCol.n_gram_fingerprint_cluster(df, "STATE" , 2).table()

STATE


cluster_size 1 (int) not nullable,cluster 2 (array<string>) nullable,count 3 (double) nullable,recommended 4 (string) nullable
8,"['Distrito⋅Federal',⋅'México⋅D.F.',⋅'DISTRITO⋅FEDERAL',⋅'Mexico⋅D.F.',⋅'Distr...",15560.0,Distrito⋅Federal


In [28]:
keyCol.n_gram_fingerprint_cluster(df, "STATE" , 2).to_json()

STATE


[{'cluster_size': 8,
  'cluster': ['Distrito Federal',
   'México D.F.',
   'DISTRITO FEDERAL',
   'Mexico D.F.',
   'Distriro Federal',
   'D.F.',
   'Estado de México',
   'distrito federal'],
  'count': 15560.0,
  'recommended': 'Distrito Federal'}]

## Distance Cluster

In [29]:
from optimus.ml import distancecluster as dc
dc.levenshtein_matrix(df,"STATE").table()

STATE_LEVENSHTEIN_1 1 (string) nullable,STATE_LEVENSHTEIN_2 2 (string) nullable,STATE_LEVENSHTEIN_DISTANCE 3 (int) nullable
mexicodf,mexicodf,0
mexicodf,distrirofederal,12
mexicodf,distritofederal,12
mexicodf,df,6
mexicodf,estadodemexico,10
distrirofederal,mexicodf,12
distrirofederal,distrirofederal,0
distrirofederal,distritofederal,1
distrirofederal,df,13
distrirofederal,estadodemexico,11


In [30]:
dc.levenshtein_filter(df,"STATE").table()

STATE_FROM 1 (string) nullable,STATE_LEVENSHTEIN_DISTANCE 2 (int) nullable,STATE_TO 3 (string) nullable
df,6,mexicodf
mexicodf,6,df
distrirofederal,1,distritofederal
distritofederal,1,distrirofederal
estadodemexico,10,mexicodf


In [31]:
df_l = dc.levenshtein_cluster(df,"STATE")

In [32]:
dc.to_json(df, "STATE")

[{'cluster': ['D.F.'], 'cluster_size': 1, 'recommended': 'D.F.', 'count': 66},
 {'cluster': ['México D.F.', 'Mexico D.F.'],
  'cluster_size': 2,
  'recommended': 'México D.F.',
  'count': 2495},
 {'cluster': ['Distriro Federal'],
  'cluster_size': 1,
  'recommended': 'Distriro Federal',
  'count': 259},
 {'cluster': ['Distrito Federal', 'DISTRITO FEDERAL', 'distrito federal'],
  'cluster_size': 3,
  'recommended': 'Distrito Federal',
  'count': 11930},
 {'cluster': ['Estado de México'],
  'cluster_size': 1,
  'recommended': 'Estado de México',
  'count': 810}]

## Feature Engineering

In [33]:
data = [('Japan', 'Tokyo', 37800000),('USA', 'New York', 19795791),('France', 'Paris', 12341418),
              ('Spain','Madrid',6489162)]
df = op.spark.createDataFrame(data, ["country", "city", "population"])

In [34]:
df.table()

country 1 (string) nullable,city 2 (string) nullable,population 3 (bigint) nullable
Japan,Tokyo,37800000
USA,New⋅York,19795791
France,Paris,12341418
Spain,Madrid,6489162


In [35]:
from optimus.ml import feature as fe

### String to Index

In [38]:
df_sti = fe.string_to_index(df, input_cols=["city", "country"])

In [39]:
df_sti.table()

country 1 (string) nullable,city 2 (string) nullable,population 3 (bigint) nullable,city_index 4 (double) not nullable,country_index 5 (double) not nullable
Japan,Tokyo,37800000,2.0,3.0
USA,New⋅York,19795791,3.0,2.0
France,Paris,12341418,0.0,1.0
Spain,Madrid,6489162,1.0,0.0


### Index to string

In [42]:
# Going back to strings from index
df_its = fe.index_to_string(df_sti, input_cols=["country_index"])

# Show DF with column "county_index" back to string
df_its.table()

country 1 (string) nullable,city 2 (string) nullable,population 3 (bigint) nullable,city_index 4 (double) not nullable,country_index 5 (double) not nullable,country_index_string 6 (string) nullable
Japan,Tokyo,37800000,2.0,3.0,Japan
USA,New⋅York,19795791,3.0,2.0,USA
France,Paris,12341418,0.0,1.0,France
Spain,Madrid,6489162,1.0,0.0,Spain


### One Hot Enconder

In [43]:
# Creating DataFrame
data = [
(0, "a"),
(1, "b"),
(2, "c"),
(3, "a"),
(4, "a"),
(5, "c")
]
df = op.spark.createDataFrame(data,["id", "category"])

# One Hot Encoding
df_ohe = fe.one_hot_encoder(df, input_cols=["id"])

# Show encoded dataframe
df_ohe.table()

id 1 (bigint) nullable,category 2 (string) nullable,id__ENCODED 3 (vector) nullable
0,a,"OrderedDict([('type',⋅0),⋅('size',⋅5),⋅('indices',⋅[0]),⋅('values',⋅[1.0])])"
1,b,"OrderedDict([('type',⋅0),⋅('size',⋅5),⋅('indices',⋅[1]),⋅('values',⋅[1.0])])"
2,c,"OrderedDict([('type',⋅0),⋅('size',⋅5),⋅('indices',⋅[2]),⋅('values',⋅[1.0])])"
3,a,"OrderedDict([('type',⋅0),⋅('size',⋅5),⋅('indices',⋅[3]),⋅('values',⋅[1.0])])"
4,a,"OrderedDict([('type',⋅0),⋅('size',⋅5),⋅('indices',⋅[4]),⋅('values',⋅[1.0])])"
5,c,"OrderedDict([('type',⋅0),⋅('size',⋅5),⋅('indices',⋅[]),⋅('values',⋅[])])"


### Vector assembler

In [44]:
# Import Vectors
from pyspark.ml.linalg import Vectors

# Creating DataFrame
data = [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)]

df = op.spark.createDataFrame(data,["id", "hour", "mobile", "user_features", "clicked"])

# Assemble features
df_va = fe.vector_assembler(df, input_cols=["hour", "mobile", "user_features"])


# Show assembled df
print("Assembled columns 'hour', 'mobile', 'user_features' to vector column 'features'")
df_va.select("features", "clicked").table()

Assembled columns 'hour', 'mobile', 'user_features' to vector column 'features'


features 1 (vector) nullable,clicked 2 (double) nullable
"OrderedDict([('type',⋅1),⋅('values',⋅[18.0,⋅1.0,⋅0.0,⋅10.0,⋅0.5])])",1.0


### Normalizer

In [45]:
# Import Vectors
from pyspark.ml.linalg import Vectors

data = [
(0, Vectors.dense([1.0, 0.5, -1.0]),),
(1, Vectors.dense([2.0, 1.0, 1.0]),),
(2, Vectors.dense([4.0, 10.0, 2.0]),)
]

df = op.spark.createDataFrame(data,["id", "features"])

df_norm = fe.normalizer(df, input_cols=["features"], p=2.0)

df_norm.table()

TypeError: isinstance() arg 2 must be a type or tuple of types